# Building an index

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
from functools import reduce
from math import log10
import re

#import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

import pandas as pd
from numpy import arange
from nltk.util import ngrams    
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize, stem
from IPython.display import Markdown, display, HTML

%matplotlib inline
default_stopwords = set(stopwords.words('portuguese'))

In [2]:
def split_spread_words(words, delim):

    """ Split then spread alpha word with certain delimiters.

    Split words with alphabetical characters that have certain 
    delimiters then spread the resulting words across the corpus.

    :param list corpus: list of words.
    :param str delim: target delimiter.

    :return: updated list of words 

    :rtype: list
    """
    
    new_words = []
    for word in words:
        if any(c.isalnum() for c in word):
            new_words.extend(word.split(delim))
        else:
            new_words.append(word)

    return new_words


def melt_merge_zipf_dfs(df_orig, df_stem, data_grain):

    """ Melt then merge zipf stats dfs with and without stemming.

    Melt two dfs containing zipf statistics generated with and
    without stemming and merge both into a single dataframe.
    
    :param pandas.core.frame.DataFrame df_orig: zipf df without stemming.
    :param pandas.core.frame.DataFrame dif_stem: zipf df with stemming
    :param str data_gram: name of the n-gram (e.g bigram).

    :return: resulting df containing both given dfs. 

    :rtype: pandas.core.frame.DataFrame
    """
    
    melt_df_orig = df_orig.melt(id_vars=[data_grain, 'Freq.','ln(Pr)'], var_name = "ranking",
                          value_vars=['ln(r)', 'ln(pred_r)'])
    melt_df_orig["stemming"] = "no_stemming"
    
    melt_df_stem = df_stem.melt(id_vars=[data_grain, 'Freq.','ln(Pr)'], var_name = "ranking",
                               value_vars=['ln(r)', 'ln(pred_r)'])
    melt_df_stem["stemming"] = "stemming"


    melt_df = pd.concat([melt_df_orig, melt_df_stem])

    return melt_df

## Load Data

In [3]:
data = pd.read_csv("../output/results.csv")
data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


### Tokenize and Filter text

In [4]:
data["words"] = data["text"].apply(lambda x: word_tokenize(x))

# Remove words that don't have at least one alphabetical character 
data["words"] = data["words"].apply(lambda words: list((word for word in words if any(c.isalnum() for c in word))))

# Remove hyphen at end of word
data["words"] = data["words"].apply(lambda words: list(word[:-1] if word[-1] == '-' else word for word in words))

# Split words joined by en dash
data["words"] = data["words"].apply(lambda words: list(word for line in words for word in line.split('–')))
# different encoding 
data["words"] = data["words"].apply(lambda words: list(word for line in words for word in line.split('—')))

# Split words joined by dot if they are alphabetical
data["words"] = data["words"].apply(lambda words: split_spread_words(words, '.'))

# Remove lone punctuation from the splits
data["words"] = data["words"].apply(lambda words: list(word for word in words if any(c.isalnum() for c in word)))

# Remove stopwords
data["words"] = data["words"].apply(lambda words: list(word for word in words if word.lower() not in default_stopwords))

In [5]:
data.head()

,title,subtitle,author,date,section,text,url,words
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...,"[juíza, federal, Ivani, Silva, Luz, Brasília, ..."
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...,"[Menos, 24, horas, juíza, federal, Ivani, Silv..."
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...,"[determinou, 31, março, 1964, estratégia, polê..."
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[pequeno, Luisa, Gabriel, preocupavam, menino,..."
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[Desde, manhã, última, segunda-feira, sucessos..."


## Inverted Index with Frequency

In [6]:
inverted_index = {} 
for doc_id, text in enumerate(data["words"]):
    fdist = FreqDist(text)
    for word in fdist:
        freq = fdist[word]
        if word not in inverted_index:
            inverted_index[word] = []
        
        inverted_index[word].append((doc_id,freq))           

In [71]:
rows = []
columns = ["word","doc_id:freq"]
for word in inverted_index:
    row = [word, inverted_index[word]]
    rows.append(row)

index_df = pd.DataFrame(rows, columns=columns)
display(Markdown("***"))
display(Markdown("### Resulting Inverted Index"))
display(HTML(index_df.sample(15).to_html(index=False)))
display(Markdown("***"))

***

### Resulting Inverted Index

word,doc_id:freq
criativo,"[(35, 2), (150, 1)]"
intensa,"[(42, 1), (131, 1), (196, 1), (207, 1), (216, 1)]"
mude,"[(77, 1), (221, 1), (242, 1)]"
apaixonou,"[(194, 1)]"
880,"[(183, 1)]"
Kirchner,"[(95, 1), (168, 1)]"
levá-las,"[(248, 1)]"
antigo,"[(36, 1), (74, 1), (114, 1), (167, 1), (173, 1), (216, 1), (237, 1), (239, 1)]"
relaxamento,"[(242, 1)]"
Vasco,"[(49, 3), (75, 3), (119, 1)]"


***

In [8]:
index_df.to_csv("../output/inverted_index.csv", index=False)

# Retrieval Approaches

## Implementation

### Document At A Time Retrieval

In [9]:
def retrieve_by_doc(index, query, k):
    try:
        import Queue as Q  # ver. < 3.0
    except ImportError:
        import queue as Q

    result = []
    q = Q.PriorityQueue()
    L = index.loc[lambda df: df.word.isin(query)]

    all_docs = set(index["doc_id:freq"].\
                   apply(lambda pairs: list(doc_id for doc_id, freq in pairs)).sum())
    
    base_documents = L["doc_id:freq"].sum()
    
    for doc in all_docs:
        score = base_documents
        
        if score == 0:
            pass
        
        elif score == []:
            score = 0
        else:
            score = [fq for d_id, fq in score if d_id == doc]
            if score != []:
                score = reduce(lambda a,b : a+b,score)
            else:
                score = 0
                
        q.put(((-1) * score,doc))
    
    i = 0
    while not q.empty() and i < k:
        pair = q.get()
        pair = ((-1) *pair[0],pair[1])
        result.append(pair)
        i += 1
        
    return result

### Term At A Time Retrieval

In [10]:
def retrieve_by_term(index, query, k):
    try:
        import Queue as Q  # ver. < 3.0
    except ImportError:
        import queue as Q

    A = {}
    result = []
    R = Q.PriorityQueue()
    q = Q.PriorityQueue()
    L = index.loc[lambda df: df.word.isin(query)]
    
    if not L.empty: 
        for doc_id, freq in L["doc_id:freq"].sum():
            if doc_id not in A:
                A[doc_id] = 0

            A[doc_id] += freq

        for doc_id, score in A.items():
            q.put(((-1) * score,doc_id))    

        i = 0
        while not q.empty() and i < k:
            pair = q.get()
            pair = ((-1) *pair[0],pair[1])
            result.append(pair)
            i += 1
    
    return result

## Placing some queries

Let's place 5 queries to each algorithm (with k = 10)

In [11]:
k = 10

In [47]:
queries_df = pd.DataFrame(["juíza","Instagram","reprodução","real","hostil"],columns=["word"])
queries_df["word"] = queries_df["word"].apply(lambda x: [x])
queries_df["by_doc"] = queries_df["word"].apply(lambda w: retrieve_by_doc(index_df, w, k))
queries_df["by_term"] = queries_df["word"].apply(lambda w: retrieve_by_term(index_df, w, k))

In [58]:
pd.set_option('max_colwidth', -1)
display(Markdown("***"))
display(Markdown("### Result by Algorithm"))
display(HTML(queries_df.to_html(index=False)))
display(Markdown("***"))

***

### Result by Algorithm

word,by_doc,by_term
[juíza],"[(2, 0), (1, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9)]","[(2, 0), (1, 1)]"
[Instagram],"[(1, 85), (1, 110), (1, 151), (1, 229), (1, 242), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]","[(1, 85), (1, 110), (1, 151), (1, 229), (1, 242)]"
[reprodução],"[(2, 160), (1, 88), (1, 123), (1, 140), (1, 196), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]","[(2, 160), (1, 88), (1, 123), (1, 140), (1, 196)]"
[real],"[(3, 213), (2, 30), (2, 85), (2, 137), (2, 188), (1, 18), (1, 25), (1, 27), (1, 36), (1, 40)]","[(3, 213), (2, 30), (2, 85), (2, 137), (2, 188), (1, 18), (1, 25), (1, 27), (1, 36), (1, 40)]"
[hostil],"[(1, 94), (1, 216), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]","[(1, 94), (1, 216)]"


***

## Is the implementation correct?

Comparing the results of both algorithms for each *single term query* side by side we can see that results are consistent regarding:
* Documents indexed by query words: 
    + The results are an exact match for both algorithms (see the results for the word **real**)
* Documents not indexed by query words:
    + Are retrievied by the algorithm **by_doc** with zero score and fill the result when not enough documents indexed by query words are found.
    + Are not retrieved by the algorithm **by_term**

To further provide evidence let's manually check the score for the *single term query* **juíza**:

In [72]:
index_df.loc[index_df['word'] == 'criativo']

,word,doc_id:freq
7127,criativo,"[(35, 2), (150, 1)]"


Looking at the dataframe we see that there are only two documents that contain the word "criativo":

* The approach 'retrieve by document' iterates over all documents so the documents that don't contain "criativo" at all would be retrieved with 0 score.

* The approach 'retrieve by document' only iterates over the documents that contain the term so the documents that don't contain it would be ignored

Taking the aforementioned in account this would give us something like (sorted by score):

* retrieve by document: [(2,35),(1,150),(0, doc_id1), (0, doc_id2) ...]
       + where we would have as many documents with 0 score as necessary to reach the parameter *k*
       
* retrieve by document: [(2,35),(1,150)]

In [81]:
display(Markdown("##### So, for k = 2 we should have identical results:"))
display(Markdown("###### (we should see *[(2,35),(1,150)]* for both)"))
print("retrieve by term, k=2: {}".format(retrieve_by_term(index_df, ["criativo"], 2)))
print("retrieve by doc, k=2: {}".format(retrieve_by_doc(index_df, ["criativo"], 2)))

display(Markdown("##### So, for k = 5:"))
display(Markdown("###### (we should see three docs with 0 score only for *by document*)"))
print("retrieve by term, k=2: {}".format(retrieve_by_term(index_df, ["criativo"], 5)))
print("retrieve by doc, k=2: {}".format(retrieve_by_doc(index_df, ["criativo"], 5)))

##### So, for k = 2 we should have identical results:

###### (we should see *[(2,35),(1,150)]* for both)

retrieve by term, k=2: [(2, 35), (1, 150)]
retrieve by doc, k=2: [(2, 35), (1, 150)]


##### So, for k = 5:

###### (we should see three docs with 0 score only for *by document*)

retrieve by term, k=2: [(2, 35), (1, 150)]
retrieve by doc, k=2: [(2, 35), (1, 150), (0, 0), (0, 1), (0, 2)]


### Comparison in terms of time and memory 

In [112]:
from ast import literal_eval

df = pd.read_csv("../output/inverted_index.csv")
df["doc_id:freq"] = df["doc_id:freq"].apply(lambda x: list(literal_eval(x)))

all_words = list(df["word"].values)